In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [25]:
import os
import io
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload


SCOPES = ["https://www.googleapis.com/auth/classroom.courses.readonly",
          "https://www.googleapis.com/auth/classroom.student-submissions.me.readonly",
          "https://www.googleapis.com/auth/classroom.announcements.readonly",
          "https://www.googleapis.com/auth/classroom.courseworkmaterials.readonly",
          "https://www.googleapis.com/auth/drive.readonly",]

def authenticate() :
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            print("credentials loaded")
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            print("verification initiated")
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            print("credentials saved")
            pickle.dump(creds, token)
    
    return creds

In [4]:
def list_courses(service):
    """Fetch and return a list of all courses."""
    courses = []
    page_token = None

    while True:
        response = service.courses().list(pageToken=page_token).execute()
        courses.extend(response.get('courses', []))
        page_token = response.get('nextPageToken', None)
        if not page_token:
            break

    return courses

def list_course_work(service, course_id):
    """Fetch and return a list of course work for a given course."""
    course_work = []
    page_token = None

    while True:
        response = service.courses().courseWork().list(courseId=course_id, pageToken=page_token).execute()
        course_work.extend(response.get('courseWork', []))
        page_token = response.get('nextPageToken', None)
        if not page_token:
            break

    return course_work

def list_drive_files(service, course_work_material):
    """Fetch and return a list of drive files for a given course work material."""
    drive_files = []

    for material in course_work_material.get('materials', []):
        if 'driveFile' in material:
            drive_files.append(material['driveFile']['driveFile'])

    return drive_files

def main():
    creds = authenticate()
    classroom_service = build('classroom', 'v1', credentials=creds)
    drive_service = build('drive', 'v3', credentials=creds)

    courses = list_courses(classroom_service)

    for course in courses:
        print(f"Course: {course['name']}")

        course_work_list = list_course_work(classroom_service, course['id'])
        for course_work in course_work_list:
            print(f"  Course Work: {course_work['title']}")

            drive_files = list_drive_files(drive_service, course_work)
            for drive_file in drive_files:
                print(f"    Drive File: {drive_file['title']} (ID: {drive_file['id']})")

if __name__ == '__main__':
    main()

verification initiated
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=12410155475-nd3gqe62s4ef8usngtrl2optm7gr35mq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A41067%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.courses.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.student-submissions.me.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.announcements.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fclassroom.courseworkmaterials.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=BiKX5yyKHauU4621d5MpDnJVyDDXw0&access_type=offline
Opening in existing browser session.
credentials saved
Course: Computer Networks S6AD
Course: 100004/IT600F COMPREHENSIVE COURSEWORK
Course: S6 AD(2021-2025) TOPOLOGICAL DATA ANALYSIS
  Course Work: ASSIGNMENT 2
    Drive File: ASSIGNMENT-2.pdf (ID: 1pRs0_SDWXjxjJn7gWLSOV2IRu6pM-N_i)
  Course Work: ASSIG

KeyboardInterrupt: 

In [5]:
def list_courses(service):
    """Fetch and return a list of all courses."""
    courses = []
    page_token = None

    while True:
        response = service.courses().list(pageToken=page_token).execute()
        courses.extend(response.get('courses', []))
        page_token = response.get('nextPageToken', None)
        if not page_token:
            break

    return courses

def list_announcements(service, course_id):
    """Fetch and return a list of announcements for a given course."""
    announcements = []
    page_token = None

    while True:
        response = service.courses().announcements().list(courseId=course_id, pageToken=page_token).execute()
        announcements.extend(response.get('announcements', []))
        page_token = response.get('nextPageToken', None)
        if not page_token:
            break

    return announcements

def list_materials(service, course_id):
    """Fetch and return a list of materials for a given course."""
    materials = []
    page_token = None

    while True:
        response = service.courses().courseWorkMaterials().list(courseId=course_id, pageToken=page_token).execute()
        materials.extend(response.get('courseWorkMaterial', []))
        page_token = response.get('nextPageToken', None)
        if not page_token:
            break

    return materials

def list_drive_files(material):
    """Fetch and return a list of drive files for a given material."""
    drive_files = []

    for item in material.get('materials', []):
        if 'driveFile' in item:
            drive_files.append(item['driveFile']['driveFile'])

    return drive_files

def main():
    creds = authenticate()
    classroom_service = build('classroom', 'v1', credentials=creds)
    drive_service = build('drive', 'v3', credentials=creds)

    courses = list_courses(classroom_service)

    for course in courses:
        print(f"Course: {course['name']}")

        announcements = list_announcements(classroom_service, course['id'])
        for announcement in announcements:
            print(f"  Announcement: {announcement['text']}")

            drive_files = list_drive_files(announcement)
            for drive_file in drive_files:
                print(f"    Drive File: {drive_file['title']} (ID: {drive_file['id']})")

        materials = list_materials(classroom_service, course['id'])
        for material in materials:
            print(f"  Material: {material['title']}")

            drive_files = list_drive_files(material)
            for drive_file in drive_files:
                print(f"    Drive File: {drive_file['title']} (ID: {drive_file['id']})")

if __name__ == '__main__':
    main()

credentials loaded
Course: Computer Networks S6AD
  Announcement:  module 5 

 PPT
    Drive File: Module 5.pptx (ID: 1lkz4qLva7bv372uuFs_BdF7xzADAellt)
  Announcement: Module 3
IPv4 and IPv6
    Drive File: IP CNN.pptx (ID: 1DdlYqI3ZqhEaUQNV0ErVFa7tGyATYhHv)
  Announcement: Module 3
Routing Algorithms
1. Distance Vector Routing
2. Link State Routing
    Drive File: DISTANCE VECTOR ROUTING.pptx (ID: 1ezhgpPeJQcWWdaej-6GSnCB0qoXkYZeq)
    Drive File: LINK STATE ROUTING.pptx (ID: 14WNsHRTkvp9gHJRkaQJveNgC8muNalos)
  Announcement: Module 4 UDP
    Drive File: UDP.pptx (ID: 1_PDJWS6Rk8aQhYTLTuuBFknKOjZt4vS9)
  Announcement: Dear Students,
Module 4  PPT part 1
    Drive File: Module 4 PPT part 1.pptx (ID: 12pTbOZZnEKcEG7T9qtIiFYI7jCpxplOo)
  Announcement: Module 3 PPT part 1
    Drive File: Module 3 PPT  Part 1.pptx (ID: 1A944rQJIQrYnQtEK_BguAD7mUNQJA7ya)
  Announcement: Module 2 Part 2 PPT
    Drive File: MOdule 2 part 2.pptx (ID: 1j1uHmBVSSbPcDQLZTvdc82JwiwcN6-9O)
  Announcement: Textbook

  Announcement: ML External Lab Exam Schedule:
27/6/2024 (Thursday ) : 
Batch 1 : 9:00 AM to 12:00 PM - U2108001 - U2108020   
Batch 2 : 1:00 PM to 4:00 PM - U2108021 - U2108040       
    
28/6/2024 (Friday ) : 
Batch 1 : 9:00 AM to 12:00 PM - U2108041 – U2108045,  U2108047- U2108060
Batch 2 : 1:00 PM to 4:00 PM - U2108061 - U2108071      
  
Venue : Sycamore Lab , KE Block, 3rd Floor.
  Announcement: Dear Students,
ML EXTERNAL LAB EXAM
The ML External Lab Exam is scheduled to be conducted on 27/06/2024 and 28/06/2024. A detailed schedule will be shared later.
  Announcement: Dear Students,

Sharing a sample question for Lab Exam.
1. Write a python program to implement Random Forest Classifier using titanic dataset.
a. Perform Data Preprocessing.
b. Perform Dimensionality Reduction.
c. Implement Random Forest classifier in the given titanic dataset.

All the best for your exams!!!
  Announcement: Dear Students,
ML INTERNAL LAB EXAM
The ML Internal Lab Exam is scheduled to be conducted

Course: S6 AD - Mini Project (2024)
  Announcement: Dear all,
Next week (8 April 2024), we are planning to schedule a Mini Project Evaluation. The objectives of the evaluation are
Project Proposal
Details of all functionalities in the project
Project Design
Current Status Demo/Code Verification
I will share a Google sheet to schedule your project presentation slot in concurrence with your guide.

 
  Announcement: Dear all,
Please form project groups with 3/4 members. The groups are also directed to conduct a basic study to define the project proposal for the Mini Project.
We have only limited time to complete the Mini Project.
Hence, when selecting the project title, consider the feasibility of implementation within the given time limit.
It is also recommended to define the project proposal considering your placement and the opportunity for a full-time internship.
General Guideline:
It must be a Web app-based OR Mobile app-based with DATABASE Connectivity.
You may include Data Science

Course: Artificial Intelligence Lab S5 AD (2021-2025)
  Announcement: Dear Students,
AI EXTERNAL LAB EXAM
The AI External Lab Exam is scheduled to be conducted on 7th and 8th February 2024. You can check your time slots on the RSMS Notice board.
Venue : Sycamore Lab
  Announcement: Dear Students,
Attaching lab sample questions for your reference. Try to solve by yourself.
    Drive File: Questions.docx (ID: 1uuYlomqg9ezeY-AmnYDBkz7sf7iDf35a)
  Announcement: Dear Students,

Please ensure the completion and verification of all your experiments by Wednesday through the concerned faculty. Additionally, remember to undergo the viva for all experiments by the same deadline. On 20-12-2023 (Wednesday), you are scheduled to have your Model Lab Examination, covering all experiments. Kindly prepare for this examination accordingly.

Best wishes for your preparations!!!
  Announcement: Dear Students, 
AI LAB INTERNAL LAB EXAM WILL BE ON 8/JAN/2024, 8.45 AM - 11.45 AM.
VENUE: SYCAMORE LAB
  Announc

  Announcement: BDA module V ppt and question bank is attached
    Drive File: Module-V-BDA.pptx (ID: 1eNwu5ippqBTedF3_Sn1jl87gtYqAqJ5F)
    Drive File: Module V-Questions.docx (ID: 1l3Axj35l6kccZYt96XmYvF3SuS2XvVbE)
  Announcement: Today's BDA class starts at 11.15 am
  Announcement: Module 5 notes
    Drive File: BDA Mod-5.pdf (ID: 1yV_YwAmTGVoXaDbzO5Bbp9-lG9CGjKYh)
  Announcement: BDA assignment 2:       Submission date: 7/5/2024
 1.       Write down the Hadoop installation
procedure in Linux/Windows.
2.  Composing MapReduce for
Calculations


The following
details are to be covered. Use necessary examples:


•Counting and Summing


•Sorting


•Finding Distinct Values


•Collating


•Filtering or Parsing


•Matrix Multiplication


•Relational – Algebra Operation


–Selection, projection, union,
intersection and difference


–Natural Join, Grouping and
Aggregation






Reference Text: Raj Kamal and Preeti Saxena, Big Data Analytics Introduction to Hadoop, Spark, and Machine- Learnin

...

  Announcement: question banks uploaded
    Drive File: Module 1 QB.docx (ID: 1lidLqObWKtft2NSxkhwtalLnKBdR2ZFK)
    Drive File: Module 2 QB.docx (ID: 1pyF2QrrSrvBoNUJVhEUP97kN9DdVqrkH)
    Drive File: Module 3 QB.docx (ID: 1Xha_a8JwRcgpFpb99u6ex2AtkdRgj3ZN)
  Announcement: Sessional marks that you have seen earlier is not final. upload the pending assignments and tutorials
  Announcement: Module 5 - File System - notes 
Study free space management from PPT
    Drive File: file system interface.ppt (ID: 1IcFmxVGXbXDcs3r68tJeO9Nfnphh79xU)
    Drive File: UNIX File system.pptx (ID: 1YrdoDfEHhjj49Rh-R6vtk2qQQ5QzT9D6)
    Drive File: Files.pdf (ID: 1NjFW30qMBZzU6oZi-AjlNn1LiYOnQerj)
    Drive File: Linux File System.docx (ID: 1SBF_naKSP8X087dMHt0-71CVuQFn1M84)
  Announcement: Syllabus
    Drive File: 101008IT300D-OPERATING SYSTEMS-Syllabus (1).pdf (ID: 1rGAAbiGOqSMKa2SW7h-DTq_x3Oidh06b)
  Announcement: Question Bank
    Drive File: Module 5 QB.docx (ID: 1m2cSEpRaaXN-u2uu6XdkyRoiG0T_he98)
 

Course: S2 AI & DS Mathematics
  Announcement: FIRST AND SECOND INTERNAL QP WITH ANSWER KEY
    Drive File: ANSWER KEY 2.pdf (ID: 1W_YKK1n7xn80s6P7-TNp2rbcN8jDAbqP)
    Drive File: ANSWER KEY.pdf (ID: 1XJhabztSjHWB3hUOtQQbZMM90SVWbd_L)
    Drive File: QP 2.pdf (ID: 15ArhkED20uYSNIF0Nb3ewMyLCUxnkw6E)
    Drive File: QP.pdf (ID: 1qQLsURc28upxKoN9TDNjef1bf9fYL4qF)
  Announcement: PREVIOUS YEAR QP AND ANSWER KEY
    Drive File: ANSWER KEY PREV QP.pdf (ID: 1TIudvqGcrgR4LfeFqBfzrFEjUIth4B1y)
    Drive File: PREV QP.pdf (ID: 1fhJX-qYFL9U7IbsFco-g8RlBrIPCfwvh)
  Announcement: HANDOUT
    Drive File: VCDT.pdf (ID: 1slAN3Uri52upXhYlczcQpJcTcv86Q9UR)
  Announcement: Tomorrow you have to join Mathematics Google classroom for attending third hour.
  Announcement: Please bring your Maths Tutorial book
  Announcement: Today you have to join meet link in Mathematics Google Classroom for attending  6th hour class.
  Announcement: Remedial class
  Announcement: Tomorrow 5 th hour will be Tutorial hour.Y

Course: S2 AIDS 2021-2025
  Material: Previous year Question papers
    Drive File: Jan 2022.pdf (ID: 1ikbH7m2LfHU3ALSicU2Vhb3XGPk1mRbY)
    Drive File: March April 2022.pdf (ID: 1Wq16OA79TF_K5u7LpL1a06qu9vEtURq8)
    Drive File: Nov 2021.pdf (ID: 1p8aijMKXi-ml0w7cMUZhwboKsLJXkp3a)
    Drive File: S2 September - October 2021.pdf (ID: 1YRec5Gf5OIhn3XgsrUu-mBQb89c2zgQ3)
    Drive File: april May 2021.pdf (ID: 1AazVDC9LtBKuqvNE0RySQuKZPakzXl3x)
  Material: Previous QP and VS 
    Drive File: 2-BEE_Valuation scheme_March2022.pdf (ID: 1BXKLJ_PETY9GDDE0ljsI5tbmoAQtxYhb)
    Drive File: 1-BEE_Regular_March2022.pdf (ID: 1zAz9XrwS0c0pJCwter7p8rLqerIt-noy)
  Material: Module 6 notes
    Drive File: AM,FM COMMUNICATION.pptx (ID: 1UvNfx31YcwEmrGIpRwA56KVBd9gwcEfN)
    Drive File: Mobile Communication.pptx (ID: 1iNk3_yev1qzqQDZEWW8AWcu9iFDAEEIe)
    Drive File: Superheterodyne receiver.pptx (ID: 1is3XGvnHkyDIDcwoMKN--yWBr-qgYJlu)
  Material: 26-Three Phase ac generation and Star-Delta connection
  

Course: RSET S2 AI& DS- PROFESSIONAL COMMUNICATION
  Announcement: TEST 5


  Announcement: TEST 4
  Announcement: TEST 3
  Announcement: TEST 2
  Announcement: TEST 1- Kindly fill in the form within 10 minutes. 
  Announcement: PFA the trial test. You will have 5 minutes to check this out. In case of any trouble, please send me a text. 
  Announcement: Dear Students,

Kindly go through the instructions before the listening test.

Regards,
Sangeeta
    Drive File: Listening Test Instructions.pdf (ID: 1IdCBKCaBoSbZPJZOmAhpmR-L52xlWdXp)
  Announcement: Dear students,

PFA some links to help you with grammar- SVA, active and passive voice, conditionals and reported speech. 
  Announcement: Dear Students,

Kindly go through the attached documents and submit the report by 5th August 2022. It has to be handwritten. I will not accept any late submissions under any circumstances. You can take a printout of the cover page and attach in the front of your handwritten report.
    Drive File: Forma

KeyError: 'title'

In [27]:
def list_courses(service):
    """Fetch and return a list of all courses."""
    courses = []
    page_token = None

    while True:
        response = service.courses().list(pageToken=page_token).execute()
        courses.extend(response.get('courses', []))
        page_token = response.get('nextPageToken', None)
        if not page_token:
            break

    return courses

def list_announcements(service, course_id):
    """Fetch and return a list of announcements for a given course."""
    announcements = []
    page_token = None

    while True:
        response = service.courses().announcements().list(courseId=course_id, pageToken=page_token).execute()
        announcements.extend(response.get('announcements', []))
        page_token = response.get('nextPageToken', None)
        if not page_token:
            break

    return announcements

def list_materials(service, course_id):
    """Fetch and return a list of materials for a given course."""
    materials = []
    page_token = None

    while True:
        response = service.courses().courseWorkMaterials().list(courseId=course_id, pageToken=page_token).execute()
        materials.extend(response.get('courseWorkMaterial', []))
        page_token = response.get('nextPageToken', None)
        if not page_token:
            break

    return materials

def list_drive_files(material):
    """Fetch and return a list of drive files for a given material."""
    drive_files = []

    for item in material.get('materials', []):
        if 'driveFile' in item:
            drive_files.append(item['driveFile']['driveFile'])

    return drive_files

def download_drive_file(service, file_id, file_name):
    """Download a file from Google Drive."""
    print(f"downloading file : {file_name}")
    request = service.files().get_media(fileId=file_id)
    file_io = io.FileIO(file_name, 'wb')
    downloader = MediaIoBaseDownload(file_io, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        print(f"Download {file_name} {int(status.progress() * 100)}%.")
        
def filter_drive_files(drive_file) :
    return drive_file["title"].split(".")[-1] in ("ppt", "pptx", "docx", "pdf")

def main():
    creds = authenticate()
    classroom_service = build('classroom', 'v1', credentials=creds)
    drive_service = build('drive', 'v3', credentials=creds)

    courses = list_courses(classroom_service)
    
    counter = 0

    for course in courses:
        print("*"*100)
        print(f"Course: {course['name']}")
        print("-"*100)

        announcements = list_announcements(classroom_service, course['id'])
        all_files = []
        for announcement in announcements:
#             print(f"  Announcement: {announcement['text']}")

            drive_files = list_drive_files(announcement)
            drive_files = list(filter(filter_drive_files, drive_files))
            for drive_file in drive_files[:5]:
                print(f"    Drive File: {drive_file['title']} (ID: {drive_file['id']})")
#                     download_drive_file(drive_service, drive_file['id'], drive_file['title'])
                if counter < 5 :
                    all_files.append((drive_file['id'], drive_file['title']))
                    counter += 1
        
        for file in all_files :
            download_drive_file(drive_service, file[0], file[1])
                
        print("-"*100)

        materials = list_materials(classroom_service, course['id'])
        for material in materials:
#             print(f"  Material: {material['title']}")

            drive_files = list_drive_files(material)
            drive_files = list(filter(filter_drive_files, drive_files))
            for drive_file in drive_files:
                print(f"    Drive File: {drive_file['title']} (ID: {drive_file['id']})")
#                 download_drive_file(drive_service, drive_file['id'], drive_file['title'])
        print("\n"*2)

if __name__ == '__main__':
    main()

credentials loaded
****************************************************************************************************
Course: Computer Networks S6AD
----------------------------------------------------------------------------------------------------
    Drive File: Module 5.pptx (ID: 1lkz4qLva7bv372uuFs_BdF7xzADAellt)
    Drive File: IP CNN.pptx (ID: 1DdlYqI3ZqhEaUQNV0ErVFa7tGyATYhHv)
    Drive File: DISTANCE VECTOR ROUTING.pptx (ID: 1ezhgpPeJQcWWdaej-6GSnCB0qoXkYZeq)
    Drive File: LINK STATE ROUTING.pptx (ID: 14WNsHRTkvp9gHJRkaQJveNgC8muNalos)
    Drive File: UDP.pptx (ID: 1_PDJWS6Rk8aQhYTLTuuBFknKOjZt4vS9)
    Drive File: Module 4 PPT part 1.pptx (ID: 12pTbOZZnEKcEG7T9qtIiFYI7jCpxplOo)
    Drive File: Module 3 PPT  Part 1.pptx (ID: 1A944rQJIQrYnQtEK_BguAD7mUNQJA7ya)
    Drive File: MOdule 2 part 2.pptx (ID: 1j1uHmBVSSbPcDQLZTvdc82JwiwcN6-9O)
    Drive File: Data-Communications-and-Networking-By-Behrouz-A.Forouzan.pdf (ID: 1qovmNQKYGOiQIvKnmc09_DTg_1AB-V3K)
    Drive File: CST 303




****************************************************************************************************
Course: S6 AD - Mini Project (2024)
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
    Drive File: 100008IT622T MINI PROJECT_Syllabus.docx (ID: 1AoQaRwMesuyFCEHK2ijgqEJgvidqJrfF)
    Drive File: 100008IT622TMINI PROJECT_CoursePlan.docx (ID: 1nADyqzIjepKaEPSYdbJkvxwXj5ejkww8)



****************************************************************************************************
Course: S5 AD - OOPs using JAVA
----------------------------------------------------------------------------------------------------
    Drive File: Module 2_Part1.pptx (ID: 1lyCR3jlABwMoIRG-SuVvClH9zDAgBudX)
    Drive File: Object Oriented Programming.pptx (ID: 1H8K8fLR6IWQbcbxxFTYVNOIf1IXTbxqO)
    Drive File: ADD_ON.docx (ID: 12_HIk9ekLVxcNZQJIF2vzfzUMAVX5

    Drive File: Module-V-BDA.pptx (ID: 1eNwu5ippqBTedF3_Sn1jl87gtYqAqJ5F)
    Drive File: Module V-Questions.docx (ID: 1l3Axj35l6kccZYt96XmYvF3SuS2XvVbE)
    Drive File: BDA Mod-5.pdf (ID: 1yV_YwAmTGVoXaDbzO5Bbp9-lG9CGjKYh)
    Drive File: Big Data Analytics-Module3,4 question bank.docx (ID: 1_B62kud6IGKhGnKqnM9Zo1UzJSDYA-dd)
    Drive File: Module-III-BDA.pptx (ID: 1rQJRRx4CZO2NGFNVHLcZEPeKb2k1F8Sh)
    Drive File: Module-IV-BDA.pptx (ID: 1FxgLdE7A3ngn4zX7XoLMXxoHxoFhc5qv)
    Drive File: BDA Mod3.pdf (ID: 1Je-YmxWZ3tla4l9a4H9c2MITquvDrpqz)
    Drive File: BDA Mod4.pdf (ID: 1PSyN8NfiEM2TZUO8AMf6wOH5Saa8pclc)
    Drive File: BDA Assignment-1.docx (ID: 1U-_A3KvtYsvF4nSMhNxQtRIGh80XmX_u)
    Drive File: 02-MongoDB-query.pdf (ID: 11ez3vLX3yb_lzeII50mwzOWKKrRN6yFy)
    Drive File: 06-CassandraDB-Commands.pdf (ID: 1Y511zASCW__SutNrTCOlBCJ6-qpp9EYz)
    Drive File: Big Data Analytics-module-2 Question Bank.docx (ID: 1QMxsA0HsmLyAh_xjf9tM9bxaSpI4pIhV)
    Drive File: Module-II-BDA.pptx (ID: 1




****************************************************************************************************
Course: AD Constitution of India
----------------------------------------------------------------------------------------------------
    Drive File: 46 to 70 ASSIGNMENT II.pdf (ID: 1kciwXZ9mlkxmmOLUSVpu1FJwrBfXW17q)
    Drive File: 21 to 45 ASSIGNMENT II.pdf (ID: 1cXJuwOqddhmdefZJoHC9Fwal2Ffmul7r)
    Drive File: Assignment II 1 to 20.pdf (ID: 1Pre5yDoLGjGJjQQgGbAsYU9rSuE6X_0G)
----------------------------------------------------------------------------------------------------
    Drive File: AD ASSESSMENT SHEET.pdf (ID: 1Hy_AOYrAw7ykUrawAdUG2h1r0rV-TCdl)
    Drive File: FUNDAMENTAL DUTIES.pptx (ID: 10tCKY-mrCTrz11uXN8llZNti2EENaGJz)
    Drive File: FUNDAMENTAL RIGHTS (session 2).pptx (ID: 1uEMq-zSh32RtZJPWNkPqp5-FGHN5fwf6)
    Drive File: FUNDAMENTAL RIGHTS.pptx (ID: 1Xql0pyhxLCrlIisfIrHB8B1fy2LMKY4L)
    Drive File: Module II Fundamental Rights.docx (ID: 1wrwHD-wV4xnsMPEzdNECpT_fu

----------------------------------------------------------------------------------------------------
    Drive File: CORRELATION.pptx (ID: 1RC4VgQgypQY3yLYN9aplWcv_4Egs9_si)
    Drive File: LINEAR REGRESSION.pptx (ID: 11jPSvVujIziQtCciepcyBPxLqZt7jcoQ)
    Drive File: q-table.pdf (ID: 1FiLO-kYDPoQtL5ow0WM1ewD2FM4DIk3E)
    Drive File: chisqtab.pdf (ID: 14Rrp1crFNa9oSAeGkAS7nNDJ8PosyCPh)
    Drive File: statistics table.pdf (ID: 1QN3P-9kWV4DNyk1jRWpSxm33djsXWHOV)
    Drive File: 101008 MA400A- Inferential Statistics- Question Paper.pdf (ID: 1hQYSSorLAKt0C-h1Tg180Ruuu9eAHsyH)
    Drive File: 101008 MA400A- Inferential Statistics- Course Plan.pdf (ID: 1udfzRZPPlKqXJcl1QI-UMRYe2xh57iHg)
    Drive File: 101008 MA400A- Inferential Statistics- Syllabus.pdf (ID: 1V6a86F1A7e2h5BnEjT_7tgg_4WWjuf6N)
    Drive File: S4AD cis.pdf (ID: 1jfwg7wxnuy3cHTOZ22znfPj_uxp6vN2r)



****************************************************************************************************
Course: DBMS Lab
----------




****************************************************************************************************
Course: S3 AD STATISTICAL FOUNDATIONS FOR DATA SCIENCE
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
    Drive File: valuation scheme format_btech S3 SFD (2).docx (ID: 1JfxI8ih4ORgW4RFEMkuau6KaFJW7e4Lw)
    Drive File: STATISTICAL FOUNDATIONS FOR DATA SCIENCE (5).pdf (ID: 13HiC18p6cvkVDgvwkGaab4pqUAimD2H6)
    Drive File: Probability-Statistics-and-Que-V-Sundarapandian (2).pdf (ID: 1xfOw11AZyr3jdL9Oao6mY5jFVIGdCAZp)
    Drive File: Screening test for S3 students notice RSMS.docx (ID: 1v3zR6TQvMnyvP2NR03A5ih5wS8cdwfmM)



****************************************************************************************************
Course: S3- Minor-Basics of Robotics -101908AE304M
-------------------------------------------------------------

----------------------------------------------------------------------------------------------------



****************************************************************************************************
Course: S2 AI & DS Mathematics
----------------------------------------------------------------------------------------------------
    Drive File: ANSWER KEY 2.pdf (ID: 1W_YKK1n7xn80s6P7-TNp2rbcN8jDAbqP)
    Drive File: ANSWER KEY.pdf (ID: 1XJhabztSjHWB3hUOtQQbZMM90SVWbd_L)
    Drive File: QP 2.pdf (ID: 15ArhkED20uYSNIF0Nb3ewMyLCUxnkw6E)
    Drive File: QP.pdf (ID: 1qQLsURc28upxKoN9TDNjef1bf9fYL4qF)
    Drive File: ANSWER KEY PREV QP.pdf (ID: 1TIudvqGcrgR4LfeFqBfzrFEjUIth4B1y)
    Drive File: PREV QP.pdf (ID: 1fhJX-qYFL9U7IbsFco-g8RlBrIPCfwvh)
    Drive File: VCDT.pdf (ID: 1slAN3Uri52upXhYlczcQpJcTcv86Q9UR)
----------------------------------------------------------------------------------------------------
    Drive File: Module 4.pdf (ID: 1vqSEQwq83FV_qyIb_EfHvmnRFN_nP2Pa)
    Drive

    Drive File: A Textbook of Engineering Physics by M N Avadhanulu.pdf (ID: 19oG_3C56L0fKhjkanxtR856geliubHdB)
    Drive File: Module 2- WP_Interference.pdf (ID: 1oZ9xj9x2Uy84TZ-Ro2eqnQM6lO_1JwsA)
    Drive File: m2-wo-p1.pdf (ID: 1WS66EgsvEZW3Uh98jH06G3LWf0Yxe5Se)
    Drive File: Module 1. Oscillations and waves Part 1.pdf (ID: 1h2yd_3hynNAUQ1I6NMxKgEP4ALmaUBSS)
    Drive File: Module 1. Oscillations and waves Part 2.pdf (ID: 1YnzxMY2pwNUN2Ng19Pdd33n84Rt_nEyR)
    Drive File: Engineering Physics A_QP_November 2021.pdf (ID: 1DlI1aiu-Ct4xuB-JU6tlTKHkKpeR4Wsh)
    Drive File: Engineering_Physics_A - May 2021.pdf (ID: 1msNhcwcf8rbRp8FBMW-l8UND96coVTIH)
    Drive File: Engineering Physics A (2).pdf (ID: 1jvetl6F-XsyJEZaz89rQLpO9VMC-UuFI)
    Drive File: Engineering Physics A - Oct 2021.pdf (ID: 1QHUn_fabwGKvyMvmVX3Pj6W0WN02suir)
    Drive File: 101902 PH900B-Engineering Physics A-ModelQP.pdf (ID: 1cHgLdpuH1ZrJ-4vbqlNZ8TZ_Pst_kDhO)
----------------------------------------------------------

KeyError: 'title'

In [14]:
list(map(lambda a: a**2, [1,2,3]))

[1, 4, 9]